In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/scratch/prashantk/cache'
os.environ['HF_DATASETS_CACHE']="/scratch/prashantk/cache"

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
model_checkpoint = "facebook/mbart-large-cc25"

In [4]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("kapilrk04/codemix-en_enhi", use_auth_token=True)
metric = load_metric("sacrebleu")

Found cached dataset parquet (/scratch/prashantk/cache/kapilrk04___parquet/kapilrk04--codemix-en_enhi-8fa8773b3690eaa4/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/1 [00:00<?, ?it/s]

/tmp/ipykernel_9219/665781172.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [5]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [6]:
tokenizer.src_lang = "en-XX"
tokenizer.tgt_lang = "hi-IN"

prefix = ""

In [7]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "en-hi"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [9]:
raw_datasets = raw_datasets["train"].train_test_split(test_size=0.2)
test_datasets = raw_datasets["test"].train_test_split(test_size=0.004)

In [ ]:
raw_datasets["test"][:1000]

In [10]:
tokenized_raw_datasets = raw_datasets.map(preprocess_function, batched=True, num_proc=5)
tokenized_test_datasets = test_datasets.map(preprocess_function, batched=True, num_proc=5)

Map (num_proc=5):   0%|          | 0/1003028 [00:00<?, ? examples/s]

/home/prashantk/miniconda3/envs/mt-model-train/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/prashantk/miniconda3/envs/mt-model-train/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/prashantk/miniconda3/envs/mt-model-train/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3635: UserWa

Map (num_proc=5):   0%|          | 0/250757 [00:00<?, ? examples/s]

/home/prashantk/miniconda3/envs/mt-model-train/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/prashantk/miniconda3/envs/mt-model-train/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/prashantk/miniconda3/envs/mt-model-train/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3635: UserWa

Map (num_proc=5):   0%|          | 0/249753 [00:00<?, ? examples/s]

/home/prashantk/miniconda3/envs/mt-model-train/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/prashantk/miniconda3/envs/mt-model-train/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/prashantk/miniconda3/envs/mt-model-train/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3635: UserWa

Map (num_proc=5):   0%|          | 0/1004 [00:00<?, ? examples/s]

/home/prashantk/miniconda3/envs/mt-model-train/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/prashantk/miniconda3/envs/mt-model-train/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/prashantk/miniconda3/envs/mt-model-train/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3635: UserWa

In [11]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/scratch/prashantk/mbart_based_en_enhi_mt_model_1",
    evaluation_strategy="steps",
    eval_steps = 5000,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=10,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=True,
    report_to="wandb",
    run_name="mbart_en_enhi_mt_model"
)


In [15]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_raw_datasets["train"],
    eval_dataset=tokenized_test_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
#trainer.train()

NameError: name 'training_args' is not defined

In [ ]:
trainer.evaluate()

In [12]:
from evaluate import TranslationEvaluator

In [19]:
type(tokenized_test_datasets["test"]["translation"])

list

In [28]:
from datasets import Dataset
testdict = {
    "text": [],
    "label":[]
}

for elem in tokenized_test_datasets["test"]["translation"]:
    testdict["text"].append(elem["en"])
    testdict["label"].append(elem["en-hi"])

    testDataset = Dataset.from_dict(testdict)

    

In [30]:
evaluator = TranslationEvaluator()
    
evaluator.compute(
    model_or_pipeline = model,
    data = testDataset,
    metric = metric,
    tokenizer = tokenizer,
    strategy = "simple"
)

KeyboardInterrupt: 